# Jaqal to IR

In this tutorial we will examine Jaqal. [Jaqal](https://www.sandia.gov/quantum/quantum-information-sciences/projects/qscout-jaqal/) (Just Another Quantum Assembly Language) is the QASM built by Sandia Labs for the [QSCOUT](https://www.sandia.gov/quantum/quantum-information-sciences/projects/qscout/) trapped-ion backend.

## JaqalPaq

We will primarily be using [JaqalPaq](https://gitlab.com/jaqal/jaqalpaq), the python package for Jaqal meta-programming. For details about JaqalPaq installation, see their [guide](https://gitlab.com/jaqal/jaqalpaq).

## Basic Usage Example

We've provided the `Sxx_circuit.jaqal` file in this repository: to confirm that your installation is valid, run the following code cell. You should see an idealized result array from the noiseless emulator: 50% |00>, 50% |11>

In [ ]:
import jaqalpaq
from jaqalpaq.parser import parse_jaqal_file
from jaqalpaq.emulator import run_jaqal_circuit
from jaqalpaq.generator import generate_jaqal_program

JaqalCircuitObject = parse_jaqal_file("Sxx_circuit.jaqal")
JaqalCircuitResults = run_jaqal_circuit(JaqalCircuitObject)
print(f"Probabilities: {JaqalCircuitResults.subcircuits[0].probability_by_str}")
JaqalProgram = generate_jaqal_program(JaqalCircuitObject)

: 

## Translating, Transpiling, and Lowering from Jaqal

In order to bring Jaqal to LLVM IR, we will leverage the built in capability of JaqalPaq to export a Jaqal program to a Qiskit circuit. Then, we can transpile, export as OpenQASM, and lower to IR using qcor as we've done previously. We will recreate the Bernstein-Vazirani circuit we've been using this far in `bernstein_vazirani.jaqal`.

In [20]:
from qiskit import QuantumCircuit
from jaqalpaq.transpilers.qiskit import ion_pass_manager, jaqal_circuit_from_qiskit_circuit


### Lets defined a new circuit of your choice with no measurements

s = '011'   # the hidden binary string
n = 3 # number of bits used to represent s


# We need a circuit with n qubits, plus one auxiliary qubit
# We also need n classical bits to write the output to
qc = QuantumCircuit(n+1, n)

# Put auxiliary in the minus state |->
# We leave the other n qubits in just |0>
qc.h(n)
qc.z(n)

# Apply Hadamard gates to each of the n "main" qubits (excluding the auxiliary)
# before querying the oracle
for i in range(n):
    qc.h(i)
    
# Apply barrier
qc.barrier()

# Apply the inner-product oracle
s = s[::-1] # reverse s to fit qiskit's qubit ordering
for q in range(n):
    if s[q] == '0':
        qc.i(q)
    else:
        qc.cx(q, n)
        
# Apply barrier
qc.barrier()

# Apply Hadamard gates after querying the oracle
for i in range(n):
    qc.h(i)


# Measurement
for i in range(n):
    qc.measure(i, i)


# This is supposed to unroll a qiskit circuit to Jaqal gates, however it struggles
# with Qiskit 'I' gate
pm = ion_pass_manager()

# In order to deal with I, we need to add a relation to the Equivalence Library
# of a pass called 'Unroll Custom Definitions'. To get at that pass:
passes_array_dict_array = pm.passes()
passes_array_dict = passes_array_dict_array[0]
passes_array = passes_array_dict['passes']
unroll_custom_defs = passes_array[0]


# Now, we get the equivalence library and add the relation
# I_Px (Jaqal Identity gate with same duration as Px) = I
equiv_lib = unroll_custom_defs._equiv_lib
equiv_lib.add_equivalence()




#pm._equiv_lib.append()
#jaq_compat = pm.run(qc)

#jaqc = jaqal_circuit_from_qiskit_circuit(jaq_compat) #qc, qiskit_to_jaq_map)
#print(generate_jaqal_program(jaqc))



['__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_basis_gates', '_equiv_lib', '_hash', 'is_analysis_pass', 'is_transformation_pass', 'name', 'preserves', 'property_set', 'requires', 'run']
